<a href="https://www.kaggle.com/code/stakum/transformer-encoder?scriptVersionId=226520929" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    # d_model is dimenstionality of input tokens
    def __init__(self,d_model,maximum_sequence_length):
        super(PositionalEncoding,self).__init__()
        pe = torch.rand(maximum_sequence_length,d_model)
        token_pos = torch.arange(0, maximum_sequence_length, dtype=torch.float).unsqueeze(1)
        divisor = torch.exp(torch.arange(0,token_dimensionality, 2).float() * -(math.log(10_000.0) / token_dimensionality))
        pe[:,0::2] = torch.sin(divisor * token_pos) # 10_000^(-2i/d) * pos
        pe[:,1::2] = torch.cos(divisor * token_pos)
        self.register_buffer('pe',pe.unsqueeze(0)) #pe will be registered as a buffer, which means it will be
        #part of the module's state but won't be considered a trainable parameter

    def forward(self,x):
        return x + self.pe[:,:x.size(1)]




In [2]:
from transformers import BertTokenizer
text = [
    "Harry Potter is a wizard.",
    "He goes to Hogwarts.",
    "He fights against Voldemort."
]

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenized_text = [torch.tensor(tokenizer.encode(t)) for t in text] # These outputs are integers ranging form 0 to vocab_size -1


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## MultiHead attention

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_model, num_heads,mask=None):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.mask = mask
        # Attention head Linear layers
        self.output_trans = nn.Linear(d_model,d_model)
        self.linear_query_trans = nn.Linear(d_model,d_model)
        self.linear_key_trans = nn.Linear(d_model,d_model)
        self.linear_value_trans = nn.Linear(d_model,d_model)

    def scaled_dot_product(self, Q, K, V, mask=None):
        attention_scores = torch.matmul(Q,K.transpose(-2,-1)) / math.sqrt(self.d_k) # The division part is the Scaling part
        attention_probabilities = torch.softmax(attention_scores,dim= -1)
        output = torch.matmul(attention_probability, V)
        return output
        
    def split_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k)
    
    # after applying attention to each head separately
    # this method combines th results back into a single tensor
    # of shape (batch, seq_length,d_model)
    def combine_heads(self, x):
        batch_size,_, seq_length, d_k = x.size()
        return x.transpose(1,2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.linear_query_trans(Q))
        K = self.split_heads(self.linear_key_trans(K))
        Q = self.split_heads(self.linear_value_trans(V))
        attention_output = self.scaled_dot_product(Q, K, V,mask=self.mask)
        output = self.output_trans(self.combine_heads(attention_output))
        return output
        
        

# Endoder Position-wise Feed-Forward NN

In [4]:
class PositionWiseNN(nn.Module):
    # d_ff is the dimenstionality of the output of the inner layer of the NN
    def __init__(self, d_model, d_ff):
        super(PositionWiseNN,self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        output = self.relu(self.fc1(x))
        return self.fc2(output)

# Encoder Block

In [5]:
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderBlock, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseNN(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attention_output = self.self_attn(x, x, x)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        

## Decoder Block

In [6]:
class DecoderBlock(nn.Module):
    def __init__(self,d_model, num_heads, d_ff, dropout,mask):
        super(DecoderBlock, self).__init__()
        self.mask = mask
        self.masked_attention_block = MultiHeadAttention(d_model, num_heads,self.mask)
        self.cross_attention = MultiHeadAttention(d_model,num_heads)
        self.feed_forward = PositionWiseNN(d_model,d_ff)
    
        self.norm_after_masking = nn.LayerNorm(d_model)
        self.norm_after_cross_attention = nn.LayerNorm(d_model)
        self.norm_after_FNN = nn.LayerNorm(d_model)

    def forward(self,target_sequence,encoder_output, encoded_seq_mask, output_seq_mask):
        attention_output = self.masked_attention(target_sequence,target_sequence,target_sequence,output_seq_mask)
        x = self.norm_after_masking(x+self.dropout(masked_attention))
        attention_output = self.cross_attention(x,encoder_output,encoder_output,encoded_seq_mask)
        x = self.norm_after_cross_attention(x+self.dropout(attention_output))
        feed_forward_output = self.feed_forward(x)
        x = self.norm_after_FNN(x+self.droupout(feed_forward_output))
          
    

## Combine Encoder with Decoder

In [7]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_layers, d_ff, mas_seq_length, droupout):
        pass